## Demographic Analysis

## Datasets

## Dataset and documentation: 
https://api.census.gov/data/2020/dec/pl.html
https://www.youtube.com/watch?v=vv4MZgqgHe8

In [69]:
RACE_POP_COUNT = 'RacePopCount.json'
datasets_and_uris = {}
datasets_and_uris[RACE_POP_COUNT] = "http://api.census.gov/data/2020/dec/pl"

#ASSETS_PATH = 'RacePopCount'

### Required Packages

In [57]:
from collections import Counter
import numpy as np
import pandas as pd
import altair as alt
from altair import datum
import json
import requests
import random
import time
import re
import requests
import os
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


## Downloading Required Data

In [70]:
def createPredicatesRacePopCount(tableId, var_cnt):
  '''
  create predicates for the census data API
  base_uri - This is the base uri for the census API
  tableId - table name, which we want to download
  var_cnt - Number of variables the table has 
  '''
  county = "*" #"099,163,125"
  state= "26"
  estimate = []
  #moe = []
  for i in range(2, var_cnt+1):
    estimate.append(tableId + '_00' + str(i) + 'N')
    #moe.append(tableId + '_00' + str(i) + 'M')

  get_vars = estimate + ['NAME'] #+ moe
  predicates = {}
  predicates["get"] = ','.join(get_vars)
  predicates["for"] = "county:" + county
  predicates["in"] = "state:" + state
  
  return(predicates)


def download_data(base_uri,predicates, path, filename, headers=''):
    filepath = path + filename
    if os.path.isfile(filepath):
        print(filepath)
        print('file already exists, no need to download')
    else:
        result = requests.get(base_uri, params=predicates, headers=headers) 
        if result.status_code == 200:
          with open(filepath, 'wb') as f:
              f.write(result.content)
        else:
          print('API returned Erro: ', result.status_code, '\n', 'Downloading of data failed.',  \
                'Please check the API')

In [73]:
tableId = 'P1'
var_cnt = 9
predicates = createPredicatesRacePopCount(tableId, var_cnt)
ASSETS_PATH = ''
download_data(datasets_and_uris[RACE_POP_COUNT],predicates, ASSETS_PATH, RACE_POP_COUNT)

## Data Transformations - One Race Population Count and Two or More Races Total Population Count

In [75]:
race_pop_count_df = pd.read_json('RacePopCount.json')